# <center>**Import necessary libraries**</center>


In [820]:
import pathlib
import pandas as pd
import math
import design_functions.rebar_information as rebar_func
import importlib
importlib.reload(rebar_func)

<module 'design_functions.rebar_information' from 'c:\\Users\\adnan.a\\Documents\\GitHub\\Killa-Design-Structures-code\\design_functions\\rebar_information.py'>

## <center>Import and read the desired beam flexural and shear reinforcement excel file from ETABS</center>

In [821]:
excel_file = pathlib.Path('excel_files\example_analysis.xlsx')
initial_df = pd.read_excel(excel_file, sheet_name=None)
pd.set_option('display.max_rows', None)

## <center>Because the excel sheet has two sheets, flexural and shear, they need to be read individually</center>

## <center>Delete unnecessary rows and columns for simplicity</center>

V1 of respective df's = first two rows are deleted<br>
V2 of respective df's = unique name, negative moment, negative combo, positive moment, and positive combo columns are removed.<br>
V3 of respective df's = each unique beam is consolidated to extract information such as section size, unqiue name.<br>
V4 of respective df's = ETABs section column is removed.<br>
V5 of respective df's = V2 df but all columns are removed except bottom area reinforcement and top area reinforcement<br>
V6 of respective df's = calculation DF including 100 indices and all rebar requirements<br>
For shear, identifying columns are removed as they are not needed. Additionally, unique beams DF doesn't need to be made as they can be taken from flexural DF. Therefore V2 may be primarily used

In [822]:
v1_flexural_df = initial_df['Sheet1'].drop([0,1])
v1_shear_df = initial_df['Sheet2'].drop([0,1])

In [823]:
v2_flexural_df = v1_flexural_df.drop(['Unnamed: 2', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 8', 'Unnamed: 9'], axis=1)
v2_shear_df = v1_shear_df.drop(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 11', 'Unnamed: 12'], axis=1)

In [824]:
v3_flexural_df = v2_flexural_df.iloc[::3]

## <center>Create width and depth columns to utilise in calculations</center>

In [825]:
v2_flexural_df.reset_index(drop=True, inplace=True)

In [826]:
v3_flexural_df.insert(3, 'Width (mm)', None)
v3_flexural_df.insert(4, 'Depth (mm)', None)

In [827]:
#this function cleans the cell of unnamed: 3 column to provide the width of each respective beam.
def clean_width_dimensions(width):
    width_list = list(width) # turn string into list of individual indexes
    width_list = [el.lower() for el in width_list] #use list comprehension to turn list into lower case values
    excluded_values = ['p', 't', 'b', '-', '_', 'c', '/'] #create list of excluded indices
    v1_width_list = [ex for ex in width_list if ex not in excluded_values] #use list comprehension to return list excluding indices
    index_list = v1_width_list.index('x') #index the list to x to retrieve required width
    v2_width_list = v1_width_list[:index_list] #slice the width list to the index x
    true_width = ''.join(v2_width_list) #join the list into a string
    return int(true_width) #turn string into int so it can be used in other functions

In [828]:
v3_flexural_df.loc[:, 'Width (mm)'] = v3_flexural_df['Unnamed: 3'].apply(clean_width_dimensions)

In [829]:
#this function cleans the cell of unnamed: 3 column to provide the depth of each respective beam.
#this function follows the same steps as clean_width_dimensions function
def clean_depth_dimensions(depth):
    depth_list = list(depth)
    depth_list = [el.lower() for el in depth_list]
    excluded_values = ['p','t', 'b', '-', '_', 'c', '/']
    v1_depth_list = [ex for ex in depth_list if ex not in excluded_values]
    index_list = v1_depth_list.index('x')
    v2_depth_list = v1_depth_list[1+index_list:-4]
    true_depth = ''.join(v2_depth_list)
    return int(true_depth)

In [830]:
v3_flexural_df.loc[:, 'Depth (mm)'] = v3_flexural_df['Unnamed: 3'].apply(clean_depth_dimensions)

## <center>With the width and depth defined, remove the ETABS section, location, and rebar area columns</center>

In [831]:
v4_flexural_df = v3_flexural_df.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 7', 'Unnamed: 10'], axis=1)

In [832]:
v4_flexural_df = v4_flexural_df.rename(columns={'Unnamed: 1': 'ETABS beam ID'})
v2_shear_df = v2_shear_df.rename(columns={'Unnamed: 7': 'Shear rebar area (mm2/m)', 'Unnamed: 10': 'Shear tension rebar area (mm2/m)', 'Unnamed: 13': 'Longitudinal tension rebar area (mm2)'})

## <center>Create Dataframe to store beam reinforcement schedule</center>

In [833]:
# Create dataframe which reflects column headings
columns = pd.MultiIndex.from_tuples([
    ('Storey', ''),
    ('Etabs ID', ''),
    ('Dimensions', 'Width (mm)'),
    ('Dimensions', 'Depth (mm)'),
    ('Bottom Reinforcement', 'Left (BL)'),
    ('Bottom Reinforcement', 'Middle (B)'),
    ('Bottom Reinforcement', 'Right (BR)'),
    ('Top Reinforcement', 'Left (TL)'),
    ('Top Reinforcement', 'Middle (T)'),
    ('Top Reinforcement', 'Right (TR)'),
    ('Shear links', 'Left (H)'),
    ('Shear links', 'Middle (J)'),
    ('Shear links', 'Right (K)'),
    ('Side Face Reinforcement', 'Left'),
    ('Side Face Reinforcement', 'Middle'),
    ('Side Face Reinforcement', 'Right')
])
beam_schedule_df = pd.DataFrame(columns=columns)

## <center>For each index of the v4 respective df, populate the main beam_schedule_df with the necessary information</center>

In [834]:
beam_schedule_df['Storey'] = v4_flexural_df['TABLE:  Concrete Beam Flexure Envelope - ACI 318-19']
beam_schedule_df['Etabs ID'] = v4_flexural_df['ETABS beam ID']
beam_schedule_df['Dimensions', 'Width (mm)'] = v4_flexural_df['Width (mm)']
beam_schedule_df['Dimensions', 'Depth (mm)'] = v4_flexural_df['Depth (mm)']
beam_schedule_df.reset_index(drop=True, inplace=True)

# <center>Create v5 df for flexure, including rebar areas and torsional longitudinal rebar<center>

In [835]:
v5_flexural_df = v2_flexural_df.drop(['TABLE:  Concrete Beam Flexure Envelope - ACI 318-19', 'Unnamed: 1', 'Unnamed: 3'], axis=1)
v5_flexural_df = v5_flexural_df.rename(columns={'Unnamed: 4': 'Location', 'Unnamed: 7': 'Bottom reinforcement', 'Unnamed: 10': 'Top Reinforcement'})
v5_flexural_df.reset_index(drop=True, inplace=True)
v2_shear_df.reset_index(drop=True, inplace=True)
v5_flexural_df['Longitudinal tension rebar area (mm2)'] = v2_shear_df['Longitudinal tension rebar area (mm2)']

# <center> Create v6 df for flexure, including width and depth. This will be the df utilised for calculations.<center>

In [836]:
#create new columns
v5_flexural_df['Width (mm)'] = [1]*len(v5_flexural_df)
v5_flexural_df['Depth (mm)'] = [1]*len(v5_flexural_df)
v5_flexural_df['ETABS beam ID'] = [1]*len(v5_flexural_df)
#add new columns to beginning of dataframe, create v6
cols = ['ETABS beam ID', 'Width (mm)', 'Depth (mm)'] + [c for c in v5_flexural_df.columns if c not in ['ETABS beam ID', 'Width (mm)', 'Depth (mm)']]
v6_flexural_df = v5_flexural_df[cols]

In [837]:
v6_flexural_df.loc[:, 'Width (mm)'] = v2_flexural_df['Unnamed: 3'].apply(clean_width_dimensions)
v6_flexural_df.loc[:, 'Depth (mm)'] = v2_flexural_df['Unnamed: 3'].apply(clean_depth_dimensions)
v6_flexural_df['ETABS beam ID'] = v2_flexural_df['Unnamed: 1']
rebar_func.side_face_assessment(v6_flexural_df, 'Depth (mm)', 'Longitudinal tension rebar area (mm2)', 'Bottom reinforcement', 'Top Reinforcement')
v6_flexural_df['Rebar Count'] = v6_flexural_df['Width (mm)'].apply(rebar_func.rebar_count)
v6_flexural_df['Bottom Rebar Schedule'] = v6_flexural_df.apply(rebar_func.rebar_calc, axis=1, args=('Rebar Count', 'Bottom reinforcement'))
v6_flexural_df['Top Rebar Schedule'] = v6_flexural_df.apply(rebar_func.rebar_calc, axis=1, args=('Rebar Count', 'Top Reinforcement'))
v6_flexural_df['Side Face Clear Space (mm)'] = v6_flexural_df['Depth (mm)'].apply(rebar_func.side_face_count)
v6_flexural_df['Side Face Reinforcement Schedule'] = v6_flexural_df.apply(rebar_func.side_face_reinf, axis=1, args=('Longitudinal tension rebar area (mm2)', 'Side Face Clear Space (mm)'))

v6_flexural_df.head()

,ETABS beam ID,Width (mm),Depth (mm),Location,Bottom reinforcement,Top Reinforcement,Longitudinal tension rebar area (mm2),Rebar Count,Bottom Rebar Schedule,Top Rebar Schedule,Side Face Clear Space (mm),Side Face Reinforcement Schedule
0,B213,300,500,End-I,3,28,Side face reinforcement is not required,2,2T16,2T16,406,Not needed
1,B213,300,500,Middle,229,90,Side face reinforcement is not required,2,2T16,2T16,406,Not needed
2,B213,300,500,End-J,302,124,Side face reinforcement is not required,2,2T16,2T16,406,Not needed
3,B215,300,500,End-I,2622,2977,Side face reinforcement is not required,2,2 rows of 2T32,2 rows of 2T32,406,Not needed
4,B215,300,500,Middle,2295,2590,Side face reinforcement is not required,2,2 rows of 2T32,2 rows of 2T32,406,Not needed


# <center>With all the flexural scheduling gathered, transpose and input the information into the beam schedule DF<center>

In [838]:
#from v6 create v7
#only includes bottom, top, and side face sched

v7_flexural_df = v6_flexural_df.drop(['Width (mm)', 'Depth (mm)', 'Location', 'Bottom reinforcement', 'Top Reinforcement', 'Longitudinal tension rebar area (mm2)', 'Rebar Count', 'Side Face Clear Space (mm)'], axis=1)
v7_flexural_df[]

,ETABS beam ID,Bottom Rebar Schedule,Top Rebar Schedule,Side Face Reinforcement Schedule
0,B213,2T16,2T16,Not needed
1,B213,2T16,2T16,Not needed
2,B213,2T16,2T16,Not needed
3,B215,2 rows of 2T32,2 rows of 2T32,Not needed
4,B215,2 rows of 2T32,2 rows of 2T32,Not needed
5,B215,2 rows of 2T32,2 rows of 2T32,Not needed
6,B232,5T20,5T20,Not needed
7,B232,5T16,5T16,Not needed
8,B232,5T20,5T20,Not needed
9,B235,2T16,2T16,Not needed


In [839]:
beam_schedule_df

Storey Etabs ID Dimensions            Bottom Reinforcement              
                   Width (mm) Depth (mm)            Left (BL) Middle (B)   
0      P4     B213        300        500                  NaN        NaN  \
1      P4     B215        300        500                  NaN        NaN   
2      P4     B232        600        600                  NaN        NaN   
3      P4     B235        300        500                  NaN        NaN   
4      P4     B237        300        500                  NaN        NaN   
5      P4     B241        300        600                  NaN        NaN   
6      P4     B257        400        600                  NaN        NaN   
7      P4     B260        600        600                  NaN        NaN   
8      P4     B332        300        600                  NaN        NaN   
9      P4     B333        600        600                  NaN        NaN   
10     P4     B342        300        600                  NaN        NaN   
11     P4     B381        300        600                  NaN        NaN   
12     P4     B400        600        600                  NaN        NaN   
13     P4     B403        600        600                  NaN        NaN   
14     P4     B412        600        600                  NaN        NaN   
15     P4     B483        500        800                  NaN        NaN   
16     P4     B895        600        600                  NaN        NaN   
17     P4     B896        600        600                  NaN        NaN   
18     P4     B897        600        600                  NaN        NaN   
19     P4     B899        500        800                  NaN        NaN   
20     P4     B900        500        600                  NaN        NaN   
21     P4     B901        500        800                  NaN        NaN   
22     P4      B78        400        800                  NaN        NaN   
23     P4     B617        400        800                  NaN        NaN   
24     P4     B391        400        600                  NaN        NaN   
25     P4      B12        600        600                  NaN        NaN   
26     P4     B713        400        600                  NaN        NaN   
27     P4     B392        400       1000                  NaN        NaN   
28     P4      B35        500        800                  NaN        NaN   
29     P4     B328        300        600                  NaN        NaN   
30     P4     B590        300        600                  NaN        NaN   
31     P4     B653        200        500                  NaN        NaN   
32     P4     B712        200        500                  NaN        NaN   

              Top Reinforcement                       Shear links              
   Right (BR)         Left (TL) Middle (T) Right (TR)    Left (H) Middle (J)   
0         NaN               NaN        NaN        NaN         NaN        NaN  \
1         NaN               NaN        NaN        NaN         NaN        NaN   
2         NaN               NaN        NaN        NaN         NaN        NaN   
3         NaN               NaN        NaN        NaN         NaN        NaN   
4         NaN               NaN        NaN        NaN         NaN        NaN   
5         NaN               NaN        NaN        NaN         NaN        NaN   
6         NaN               NaN        NaN        NaN         NaN        NaN   
7         NaN               NaN        NaN        NaN         NaN        NaN   
8         NaN               NaN        NaN        NaN         NaN        NaN   
9         NaN               NaN        NaN        NaN         NaN        NaN   
10        NaN               NaN        NaN        NaN         NaN        NaN   
11        NaN               NaN        NaN        NaN         NaN        NaN   
12        NaN               NaN        NaN        NaN         NaN        NaN   
13        NaN               NaN        NaN        NaN         NaN        NaN   
14        NaN               NaN        NaN        NaN         